# Try moving M3 port with notebook

Craig Lage - 06Mar25

In [ ]:
import asyncio
import time
import os
import numpy as np
from astropy.time import Time

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

In [ ]:
logger = logging.getLogger(f"Daytime Checkout {Time.now()} UT")
logger.level = logging.DEBUG

In [ ]:
def check_evt_time(evt):
    event_time = Time(evt.private_sndStamp, format="unix", scale="tai")
    event_time.format = "iso"
    event_time = event_time.utc
    return event_time

Helper function

### Instantiate the control classes

In [ ]:
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

## M3 functionality 

M3 verification is not required every time this daytime checkout is run. Check the DayTime Checkout historical data to corroborate if this section is meant to run today.  

Confirm M3 mirror rotation is operational and its valve is routinely exercized by running the following commands. 

First, move to the zenith where the Nasmyth motor can function with ease.  

In [ ]:
start_az=.0
start_el=80.0
start_rot=0
await atcs.point_azel(az=start_az, el=start_el, rot_tel=start_rot)

Once the enumerations have been updated in the class M3ExitPort https://github.com/lsst-ts/ts_idl/blob/2add7a88bba16f2b13448c3ea6f78ef6d988ce83/python/lsst/ts/idl/enums/ATMCS.py#L34-L37, substitute the following five cells with: 

`from lsst.ts.idl.enums.ATMCS import M3ExitPort`
Move to the Nasmyth1 port
`await atcs.rem.atmcs.cmd_setInstrumentPort.set_start(port=M3ExitPort.NASMYTH1)`
and back to Nasmyth 2
`await atcs.rem.atmcs.cmd_setInstrumentPort.set_start(port=M3ExitPort.NASMYTH2)`

In [ ]:
from lsst.ts.idl.enums.ATMCS import M3ExitPort

In [ ]:
await atcs.rem.atmcs.evt_m3PortSelected.aget()

In [ ]:
m3 = await atcs.rem.atmcs.evt_m3PortSelected.aget()
m3_time = check_evt_time(m3)
logger.info(f'M3 Port selected -- {m3.selected} at {m3_time} UT')

In [ ]:
M3ExitPort = {'NASMYTH1': 1, 'NASMYTH2':2}

Move to the Nasmyth1 port

In [ ]:
await atcs.rem.atmcs.cmd_setInstrumentPort.set_start(port=M3ExitPort['NASMYTH1'])
m3 = await atcs.rem.atmcs.evt_m3PortSelected.aget()
m3_time = check_evt_time(m3)
logger.info(f'M3 Port selected -- {m3.selected} at {m3_time} UT')

and back to Nasmyth 2

In [ ]:
await atcs.rem.atmcs.cmd_setInstrumentPort.set_start(port=M3ExitPort['NASMYTH2'])
m3 = await atcs.rem.atmcs.evt_m3PortSelected.aget()
m3_time = check_evt_time(m3)
logger.info(f'M3 Port selected -- {m3.selected} at {m3_time} UT')